In [ ]:
import numpy as np
import random
import copy
import datetime
import platform
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from collections import deque
from mlagents_envs.environment import UnityEnvironment, ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
state_size = 12*4
action_size = 2

load_model = False
train_mode = True

batch_size = 128
mem_maxlen = 10000
discount_factor = 0.9
actor_lr = 1e-4
critic_lr = 5e-4
tau = 1e-3

mu = 0
theta = 1e-3
sigma = 2e-3

train_step = 300000 if train_mode else 0
save_step = 10000
test_step = 10000
train_start_step = 2000

print_interval = 10
save_interval = 100

In [ ]:
game = "Kart1.exe"
file_path = f"./{game}"
save_path = f"./"
load_path = f"./"

engine_configuration_channel = EngineConfigurationChannel()
env = UnityEnvironment(file_name=file_path,
                       side_channels=[engine_configuration_channel],
                       worker_id=1)
env.reset()
behavior_name = list(env.behavior_specs.keys())[0]
spec = env.behavior_specs[behavior_name]

episode = 10
for ep in range(episode):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1
    done = False
    ep_rewards = 0
    
    while not done: 
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        
        """action"""
        # 1. 앞으로 가는 action. [(horizontal), (vertical)]이니 앞으로 가는 action 
        action = np.array([[0, 1]], dtype=np.float32) 
        
        # 2. Random action
        #action = np.array([np.random.randint(-1, action_size, size=2)])
        
        action_tuple = ActionTuple()
        action_tuple.add_continuous(action)
        print(action)
        env.set_actions(behavior_name, action_tuple)
        env.step()
        decision_steps, terminal_steps = env.get_steps(behavior_name)